In [49]:
import sys
import warnings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from imblearn import over_sampling

IN_COLAB = 'google.colab' in sys.modules
warnings.filterwarnings("ignore", category=UserWarning)

if IN_COLAB:
  df = pd.read_csv('https://raw.githubusercontent.com/chakraskun/churn-modelling/main/Churn_Modelling.csv')
else:
  df = pd.read_csv('Churn_Modelling.csv')

In [50]:
df.sample(5)

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
2388,2389,15778395,McIntyre,762,Germany,Male,34,4,88815.56,2,1,0,68562.26,1
5724,5725,15784286,Wood,641,Spain,Male,40,5,102145.13,1,1,1,100637.07,0
6094,6095,15575623,Simpson,589,France,Female,31,10,110635.32,1,1,0,148218.86,0
6579,6580,15652401,Lafleur,496,France,Female,36,7,0.00,2,0,0,108098.28,0
4088,4089,15788536,Armit,755,Germany,Male,40,2,137430.82,2,0,0,176768.59,0


In [51]:
numerical = [
  'CreditScore',
  'Age',
  'Balance',
  'EstimatedSalary',
  'Tenure',
  'NumOfProducts',
]
categorical = [
  'Geography',
  'Gender',
  'HasCrCard',
  'IsActiveMember',
]

# Data Preprocessing

- Dari hasil EDA didapat bahwa tidak ada value yang null
- drop CustomerId, Surname dan RowNumber
- tidak ada duplikat

In [52]:
df.duplicated().sum()

0

In [53]:
try:
  df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)
except:
  pass

df.sample(5)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
4933,420,Germany,Female,31,1,108377.75,2,1,1,9904.63,0
4441,624,France,Female,38,3,0.00,2,1,1,163666.85,0
2283,702,France,Male,45,9,74989.58,1,1,1,171014.69,0
5175,699,Spain,Female,29,9,127570.93,2,1,0,164756.81,0
8157,719,Spain,Female,32,7,0.00,1,0,0,76264.27,0


# Outliers Handling

Outlier untuk sementara tidak dihandle, dikarenakan outlier bersifat statistikal saja,

# Split Dataset

In [54]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [55]:
try:
  df_test.drop(columns=['Exited'], inplace=True)
except:
  print(df_test.columns)

/home/chakras/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [56]:
df_test.to_csv('test_raw.csv', index=False)
df_train.to_csv('train_raw.csv', index=False)

# Transformation Train & Test Dataset

## Re-read raw dataset

In [57]:
test = pd.read_csv('test_raw.csv')
train = pd.read_csv('train_raw.csv')

In [58]:
handled_dataset = [train, test]

## Handle Age

In [59]:
for i in handled_dataset:
  i['LogAge'] = np.log(i['Age'])

## Handle Balance, EstimatedSalary, Tenure, NumOfProducts

In [60]:
for i in handled_dataset:
  i['LogAgeNorm'] = MinMaxScaler().fit_transform(i['LogAge'].values.reshape(-1, 1))
  i['BalanceStd'] = StandardScaler().fit_transform(i['Balance'].values.reshape(len(i), 1))
  i['BalanceNorm'] = MinMaxScaler().fit_transform(i['BalanceStd'].values.reshape(len(i), 1))
  i['EstimatedSalaryStd'] = StandardScaler().fit_transform(i['EstimatedSalary'].values.reshape(len(i), 1))
  i['EstimatedSalaryNorm'] = MinMaxScaler().fit_transform(i['EstimatedSalaryStd'].values.reshape(len(i), 1))
  i['TenureNorm'] = MinMaxScaler().fit_transform(i['Tenure'].values.reshape(len(i), 1))
  i['NumOfProductsNorm'] = MinMaxScaler().fit_transform(i['NumOfProducts'].values.reshape(len(i), 1))
  i['CreditScoreNorm'] = MinMaxScaler().fit_transform(i['CreditScore'].values.reshape(len(i), 1))

# Feature Encoding

In [61]:
mapping_gender = {
    'Female' : 0,
    'Male' : 1
  }
for i in handled_dataset:
  i['Gender'] = i['Gender'].map(mapping_gender)

In [62]:
for i in handled_dataset:
  one_hot_encoding = pd.get_dummies(i['Geography'], prefix='geo')
  if i.shape[0] == 8000:
    train = train.join(one_hot_encoding)
  if i.shape[0] == 2000:
    test = test.join(one_hot_encoding)

# Drop Unused Columns

In [63]:
handled_dataset = [train, test]
for i in handled_dataset:
  i.drop(columns=['Geography','Age', 'Balance', 'EstimatedSalary', 'EstimatedSalaryStd', 'BalanceStd', 'Tenure', 'NumOfProducts', 'LogAge', 'CreditScore'], inplace=True)

In [64]:
train.sample(5)

,Gender,HasCrCard,IsActiveMember,Exited,LogAgeNorm,BalanceNorm,EstimatedSalaryNorm,TenureNorm,NumOfProductsNorm,CreditScoreNorm,geo_France,geo_Germany,geo_Spain
5178,0,1,1,0,0.638374,0.494150,0.202401,0.3,0.333333,0.556,1,0,0
4625,1,0,1,0,0.352678,0.000000,0.732508,0.4,0.333333,0.492,0,0,1
1424,0,1,1,0,0.176339,0.380665,0.228461,0.3,0.000000,0.462,1,0,0
86,0,0,1,0,0.176339,0.458149,0.678796,0.6,0.333333,0.658,1,0,0
334,0,1,0,0,0.270828,0.000000,0.479621,0.9,0.333333,0.284,1,0,0


In [65]:
test.sample(5)

,Gender,HasCrCard,IsActiveMember,LogAgeNorm,BalanceNorm,EstimatedSalaryNorm,TenureNorm,NumOfProductsNorm,CreditScoreNorm,geo_France,geo_Germany,geo_Spain
1011,0,1,0,0.575126,0.000000,0.521352,0.1,0.333333,0.462,0,0,1
697,0,1,0,0.176339,0.000000,0.580798,0.5,0.333333,0.546,1,0,0
1950,1,1,1,0.407607,0.576634,0.346580,0.7,0.000000,0.724,1,0,0
1417,1,0,0,0.727690,0.000000,0.518991,0.9,0.333333,0.374,1,0,0
282,0,0,0,0.601213,0.540054,0.381845,0.9,0.333333,0.642,0,1,0


# Handle imbalance class on train df

In [66]:
X = train[[col for col in train.columns if (str(train[col].dtype) != 'object') and col != 'Exited']]
y = train['Exited'].values
X_over_SMOTE, y_over_SMOTE = over_sampling.SMOTE(sampling_strategy=0.5).fit_resample(X, y)

print('BEFORE')
print(pd.Series(y).value_counts())
print('----------------------')
print('AFTER')
print(pd.Series(y_over_SMOTE).value_counts())

BEFORE
0    6356
1    1644
dtype: int64
----------------------
AFTER
0    6356
1    3178
dtype: int64


In [67]:
X_over_SMOTE['Exited'] = y_over_SMOTE
X_over_SMOTE.to_csv('churn_train.csv', index=False)

In [68]:
test.to_csv('churn_test.csv', index=False)

# Kesimpulan

- Outlier pada feature `Age` dan `CreditScore` tidak dihandle, karena bersifat statistical (to be reviewed step selanjutnya)
- Pada feature `Age` dilakukan log transformation agar mendapatkan hasil mendekati distribusi normal
- Feature `Balance` dan `EstimatedSalary` dilakukan standarizarion (to be reviewed di step selanjutnya)
- Feature `Tenure` dan `NumOfProducts` dilakukan normalization (to be reviewed di step selanjutnya)
- Dataset di split menjadi 80% train dan 20% test (`random_state=42`)
- Data imbalance pada dataset train di handle dengan menggunakan oversampling SMOTE